# Extract n Documents for Storage in seperate JSON File
This notebook extracts n documents (eg n=100) from the source collection and stores the result in a seperate file (for easier review and processing).

## Get Data fom Local Storage

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
sourcePath = "/gdrive/MyDrive/GermanDataSets/CourtRulings/"
!ls {sourcePath.replace(' ', '\ ')} -lha

total 1.4G
-rw------- 1 root root 7.1M Aug 12 19:29  1k_2019-02-19_oldp_cases_textout.json.zip
-rw------- 1 root root 679M Aug 15 11:24  2019-02-19_oldp_cases_textout_1.json.zip
-rw------- 1 root root 672M Aug 12 18:59  2019-02-19_oldp_cases_textout.json.zip
-rw------- 1 root root 1.1M Aug 15 11:25  Extract_100_Docs.ipynb
-rw------- 1 root root 3.6M Aug 15 11:24 'Extract Text from Court Rulings.ipynb'


In [ ]:
!unzip {sourcePath.replace(' ', '\ ') + '2019-02-19_oldp_cases_textout_1.json.zip'} -d .

Archive:  /gdrive/MyDrive/GermanDataSets/CourtRulings/2019-02-19_oldp_cases_textout_1.json.zip
  inflating: ./File_2019-02-19_oldp_cases_textout_1.json  


# Setup Spark

In [ ]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-08-15 11:27:23--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-08-15 11:27:23--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.1.2 and Spark NLP 3.2.1
-                   100%[==

In [ ]:
from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp

spark = sparknlp.start()
spark

In [ ]:
!head -n 10 ./File_2019-02-19_oldp_cases_textout_1.json

{"id":188482,"text_Content":"\n Tenor     \n Als funktional zuständig wird die allgemeine Zivilkammer bestimmt.     \n Gründe     \n I.   \n Die in München ansässige Klägerin, ein Versicherungsunternehmen, begehrt nach Abgabe an das im Mahnbescheid als Streitgericht bezeichnete Landgericht Augsburg mit Anspruchsbegründung vom 12.2.2018 von dem im Bezirk des Landgerichts Augsburg wohnhaften Beklagten, einem Versicherungsvermittler, Rückzahlung von Provisionsvorschüssen i.H.v. 6.150,23 € wegen vom Beklagten vermittelter, aber stornierter bzw. reduzierter Versicherungen. Die Parteien schlossen am 1.4.2009 einen Versicherungsagenturvertrag, wonach der Beklagte ausschließlich für die Klägerin im Wesentlichen die Erschließung neuer Kundenkreise und die Pflege der Bestandskunden übernahm. Die Klägerin stützt ihren Anspruch auf Ziff. 4 der nach dem Agenturvertrag geltenden Allgemeinen Provisionsbedingungen, wonach eine Abschlussprovision zurückbelastet wird, soweit ein Versicherungsvertrag sto

In [ ]:
df = spark.read.json('./File_2019-02-19_oldp_cases_textout_1.json')


In [ ]:
df.show(10, truncate=False)
#df.where('text_Content is not null').show(50, truncate=False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
outFileName = './tmp_100'
df.limit(100).coalesce(1).write.format('json').save(outFileName)

In [ ]:
ls ./tmp_100 -lha

total 2.9M
drwxr-xr-x 2 root root 4.0K Aug 15 11:45 ./
drwxr-xr-x 1 root root 4.0K Aug 15 11:45 ../
-rw-r--r-- 1 root root 2.9M Aug 15 11:45 part-00000-d59a3da3-b48a-4b88-9288-3baa41dd84c4-c000.json
-rw-r--r-- 1 root root  23K Aug 15 11:45 .part-00000-d59a3da3-b48a-4b88-9288-3baa41dd84c4-c000.json.crc
-rw-r--r-- 1 root root    0 Aug 15 11:45 _SUCCESS
-rw-r--r-- 1 root root    8 Aug 15 11:45 ._SUCCESS.crc


In [ ]:
!mv ./tmp_100/part-00000-d59a3da3-b48a-4b88-9288-3baa41dd84c4-c000.json 100_2019-02-19_oldp_cases_textout.json

In [ ]:
!zip {sourcePath.replace(' ', '\ ') + '100_2019-02-19_oldp_cases_textout.json.zip'} ./100_2019-02-19_oldp_cases_textout.json

  adding: 100_2019-02-19_oldp_cases_textout.json (deflated 73%)
